In [1]:
from nltk.corpus.reader.wordnet import Lemma
import nltk
from nltk import FreqDist
nltk.download('stopwords')

# For file reading
import os
from os import listdir
# For pre-processing 
from nltk import word_tokenize
from nltk.tokenize import punkt
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
import string
import random

import numpy as np
# For skip-gram model
!pip install gensim
import gensim
from gensim.models import Word2Vec

# For Cluster
from sklearn.cluster import KMeans


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rubyli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
def read_data(path: str) -> list:
    """ Read files from a directory and then append the data of each file into a list. """
    folder = listdir(path)
    res = []
    for files in folder:
        # check if current path is a file
        if files != "README.txt":
            filePath = os.path.join(path, files)
            if os.path.isfile(filePath):
                with open(filePath, "r") as file:
                    lines = file.readlines()
                res.append(lines)
    return res

def process_document(document: str) -> list:
        """ pre-process a document and return a list of its terms: str->list"""
        
        # Remove number
        text_nonum = re.sub(r'\d+', ' ', document)
        
        pattern = r'''(?x)        # set flag to allow verbose regexps
                    (?:[A-Z]\.)+     #abbreviations
                    |\[
                    |[^\w\s]
                    |\#
                    |[-.(]+           #double hyphen, ellipsis, open parenthesis
                    |\S\w*
                    |\$?\d+(?:\.\d+)?%? #currency and percentages
        '''
        #Tokenization
        tokenList = nltk.regexp_tokenize(text_nonum, pattern)
        #To lower case
        tokenList = [word.lower() for word in tokenList]
        #Remove Punctuation
        tokenList = list(filter(lambda word: punkt.PunktToken(word).is_non_punct,tokenList))
        #Remove stopwords
        stopW = stopwords.words("english")
        stopW.append("u")
        stopW.append("p")
        # stopW.append("mp")
        tokenList = list(filter(lambda word: word not in stopW,tokenList))
        # Lemmatisation 
        lemma = WordNetLemmatizer()
        tokenList = [lemma.lemmatize(word) for word in tokenList]

        return tokenList 

def process_reviews_str(res: list) -> list:
    # merge all reviews
    doc = ""
    for a in res:
        for b in a:
            doc += b
    # Pre-process documents        
    producedDoc = process_document(doc)
    return producedDoc


def get_top50(producedDoc: str) -> list:
    # Find the 50 most frequently occurred words
    # Get the frequency of each word
    word_frequencies = FreqDist(producedDoc)
    # Sort the dictionary by frequency
    sorted_frequencies = sorted(word_frequencies.items(), key=lambda item: item[1], reverse=True)
    # Select the top 50 words
    target_words = [item[0] for item in sorted_frequencies[:50]]
    
    return target_words

def pseudowords(target_words: list, processed_corpus: list) -> list:
    # Sample half of the target words
    sample_size = len(target_words) // 2
    sample = random.sample(target_words, sample_size)
    
    
    # Create pseudowords for the sampled target words
    madeups = [word[::-1] for word in sample]
    # Replace the sampled occurrences of the target words with their pseudowords
    pseudowords = target_words[:]
    pseudo_corpus = processed_corpus[:]
    for i, word in enumerate(target_words):
        if word in sample:
            # Find the index of the word to be replace
            replacement_index = sample.index(word)
            # Replace the word with the corresponding word from the replacement list
            pseudowords[i] = madeups[replacement_index]
    return pseudowords, pseudo_corpus


In [88]:
def featureVector(processed_list: list, sample: str) -> np.array:
    
    cbow_model = Word2Vec(processed_list, min_count = 1, vector_size = 100, window = 5)
    word_vectors = cbow_model.wv.vectors
    # print(list(cbow_model.wv.key_to_index))
    
    M = []
    for s in sample:
        vec = cbow_model.wv[s]
        M.append(vec)
    
    # M = np.array(word_vectors)
    return M
#     M = []
#     for s in sample:
#         word_vectors = cbow_model.wv[s]
#         M.append(word_vectors)
        
#     print(M)


#     # svd
#     # u, s, v = np.linalg.svd(positive_skip_grams)
#     u, s, v = np.linalg.svd(word_vectors)
#     # u, s, v = np.linalg.svd(M)
#     M = np.dot(u, np.diag(s))
    
#     # norm
#     M = M / np.linalg.norm(M, axis=1)[:, None]
#     return M

def getCluster(X: np.array) -> np.array:
    # Set the number of clusters to 50
    num_clusters = 50
    
    # Create a KMeans instance with the specified number of clusters
    km = KMeans(n_clusters=50).fit(X)
    
    performance = []
    
    labels = km.labels_
    labels.tolist()
    performance.append((np.sum(labels[0:50] == labels[50:100])) / len(labels[0:50]))
    print("target words: " + str(labels[50:100]))
    print("pesudo words: " + str(labels[0:50]))
    print("Performance: " + str(performance[0]))
        
    return performance

In [89]:
# main()

In [90]:
path = "/Users/rubyli/Desktop/GitHubRepos/UoM/NLP-Distributional-Semantics-Neural-Network-for-Classifying-Product-Reviews/product_reviews"
# Reading documents
res = read_data(path)
processed_str = process_reviews_str(res)
processed_list = process_reviews_list(res)
target_words = get_top50(processed_str)

In [91]:
# pseudo_words = pseudowords(target_words)
pseudo_words, new_corpus = pseudowords(target_words, processed_str)
print(new_corpus[:10])

['krow', 'work', 'taerg', 'odor', 'us', 'regular', 'gab', 'complain', 'diaper', 'pmahc']


In [92]:
# print(target_words)

In [93]:
# print(pseudo_words)

In [94]:
# M = featureVector(target_words, pseudo_words, res)
# clust = getCluster(M)
sample = target_words + pseudo_words
M = featureVector(new_corpus, sample)

KeyError: "Key 'use' not present"

In [63]:
print(M.shape)

(5186, 100)


In [ ]:
print(clust)

In [ ]:
# p = getProbability(clust)
# print(p)